In this notebook we use a demo transcript and store it into Pinecone and retrieve data according to a query.

Author: Sartaj

In [109]:
# # Have the `transcript.csv` file in the same directory as this notebook
# # Create a .env file with the following variables:
# OPENAI_API_KEY
# PINECONE_API_KEY
# PINECONE_ENVIRONMENT

In [16]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

False

Read Data and Pre-process

In [33]:
transcript = pd.read_csv('transcript.csv')
transcript.head()

,Unnamed: 0,start_time,end_time,speaker_label,text
0,0,0.000150,0.083500,Speaker 1,How do I defend myself and my Children against...
1,1,0.083650,0.128983,Speaker 1,And you said to yourself that the police may n...
2,2,0.129167,0.135000,Speaker 1,"Tell me,"
3,3,0.135150,0.189983,Speaker 2,you know what the best chance you got in a sit...
4,4,0.190150,0.203983,Speaker 2,She just stated that


In [34]:
transcript.dropna(inplace=True)
transcript.drop(['Unnamed: 0', 'end_time' ], axis=1, inplace=True)
transcript.head()

,start_time,speaker_label,text
0,0.000150,Speaker 1,How do I defend myself and my Children against...
1,0.083650,Speaker 1,And you said to yourself that the police may n...
2,0.129167,Speaker 1,"Tell me,"
3,0.135150,Speaker 2,you know what the best chance you got in a sit...
4,0.190150,Speaker 2,She just stated that


Creating Embeddings

In [6]:
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

In [7]:
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [8]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0]) # (x,1536) 1536 is the embedding size

(2, 1536)

In [53]:
def create_embedding(text):
    return embed.embed_documents([text])[0]

# get first row of transcript
test_embedding_function = create_embedding(transcript.iloc[0]['text'])
len(test_embedding_function) # 1 X 1536

1536

Pinecone

In [ ]:
import pinecone
import time

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY') or 'PINECONE_API_KEY'
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'
index_name = 'langchain-retrieval-transcript'
namespace = 'meeting_topic'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT)

In [100]:
print(pinecone.list_indexes())
for index in pinecone.list_indexes():
    print(pinecone.delete_index(index))

[]


In [101]:
# create index
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=len(res[0])  # model_name = 'text-embedding-ada-002'; 1536 dim of text-embedding-ada-002
    )   
    
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)


In [102]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [91]:
# index.delete(delete_all=True, namespace=namespace)

{}

Inserting data into Pinecone

In [103]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 50
texts = []
metadatas = []
meeting_id = 1

for i, record in tqdm(transcript.iterrows()):
    # first get metadata fields for this record
    metadata = {
        'speaker': record['speaker_label'],
        'start_time': round(record['start_time'], 4), # limit to 4 decimal places 
        'meeting_id': meeting_id,
        'text': record['text'], # Storing the text in the metadata for now, later we'd need to decode it from vectors
    }

    record_texts = record['text']

    texts.append(record_texts)
    metadatas.append(metadata)

    # print(texts)
    # print(metadatas)
    
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas), namespace=namespace)
        texts = []
        metadatas = []
        meeting_id += 1

# add any remaining texts
if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
    
time.sleep(5)    

954it [00:18, 51.25it/s]


In [106]:
index.describe_index_stats()   

{'dimension': 1536,
 'index_fullness': 0.00954,
 'namespaces': {'': {'vector_count': 4},
                'meeting_topic': {'vector_count': 950}},
 'total_vector_count': 954}

Querying Pinecone DB

In [ ]:
# Need to look up about using LangChain for retrieval

In [108]:
query  = "What was talked regarding United States Congress?"
downstr_response = index.query(
    vector= embed.embed_documents([query])[0],
    # filter={
    #     "meeting_id": {"$in":[1, 2]}
    # },
    namespace=namespace, 
    top_k=10,
    include_metadata=True,
)
downstr_response

{'matches': [{'id': '704f8d0c-23dc-49aa-a016-dba4364fbdd2',
              'metadata': {'meeting_id': 12.0,
                           'speaker': 'Speaker 7',
                           'start_time': 27.0078,
                           'text': 'The question is more so will the United '
                                   'States of America'},
              'score': 0.829302073,
              'values': []},
             {'id': 'e93c5dad-3b61-4250-83b4-e97156f9079a',
              'metadata': {'meeting_id': 12.0,
                           'speaker': 'Speaker 7',
                           'start_time': 26.584,
                           'text': 'But I think the question is more so will '
                                   'the United States of America'},
              'score': 0.819661617,
              'values': []},
             {'id': '61d40cda-804c-4994-84ae-8fa1581e9a22',
              'metadata': {'meeting_id': 2.0,
                           'speaker': 'Narrator',
                 